In [22]:
import csv
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [ ]:
#county names: https://data.ca.gov/sites/default/files/CA_COUNTIES_0.csv
#cities and counties needed

In [23]:
## file name and setup 

file_name = 'index'
terms_of_interest = ['carcinogenic', 'arsenic']

basic_url = 'https://www.communitywatercenter.org/news?n=448&page='
n_pages = 34

In [24]:
pagelinks =[]

for number in range(1,n_pages+1):

    url = basic_url + str(number)
    page = urlopen(url)
    soup = BeautifulSoup(page, 'html.parser') #containing the HTML of the page
    name_box = soup.find_all('h3', attrs={'class': 'news_heading'})
    for count in range(0, len(name_box)):
            for link in name_box[count].find_all('a', href=True):
                pagelinks.append(link['href']) 

In [15]:
pagelinks

['/justicia_denegada',
 '/justicedelayed',
 '/julio_2018_enews',
 '/july_2018_enews',
 '/filing_period_open',
 '/how_do_i_begin_the_process',
 '/how_many_seats_are_up_for_election_in_my_community',
 '/who_is_eligible_to_join_a_water_board',
 '/waterboardelectionreport',
 '/what_do_local_water_boards_do',
 '/junio_enews',
 '/june_2018_enews',
 '/sadwfrallyjune',
 '/newdirectoroforganizing',
 '/sadwfcommit',
 '/cepupdate',
 '/may_2018_enews',
 '/an_evening_of_art_and_justice',
 '/ndwwrallies',
 '/may2018cep',
 '/sadwfupdate',
 '/2017_cep',
 '/enews_marzo',
 '/march_2018_enews',
 '/enews_febrero',
 '/2018irvineleadershipaward',
 '/2018_wjla_sponsors',
 '/budget2018',
 '/looking_back_on_2017',
 '/toxic_water_problems',
 '/2017legwrapup',
 '/volunteer_canvassers',
 '/new_cwc_members',
 '/gsp_toolkit',
 '/agua_potable_sb623',
 '/op_ed_sb_623',
 '/la_times_editorial',
 '/visaliawaterjustice2017',
 '/sb623_fight_continues',
 '/comments_submitted_for_ab_401',
 '/kvpr_sb623',
 '/united_for_justi

In [177]:
soup

 <!DOCTYPE html>

<html lang="en" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta charset="utf-8">
<meta content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no" name="viewport">
<title>Water Blog - Community Water Center</title>
<link href="https://communitywatercenter.nationbuilder.com/themes/1/56202ea52213931753000001/0/attachments/14471960131529015221/default/theme.scss" rel="stylesheet" type="text/css"/>
<!-- Latest compiled and minified CSS -->
<link crossorigin="anonymous" href="//maxcdn.bootstrapcdn.com/bootstrap/3.3.6/css/bootstrap.min.css" integrity="sha384-1q8mTJOASx8j1Au+a5WDVnPi2lkFfwwEAa8hDDdjZlpLegxhjVME1fgjWPGmkzs7" rel="stylesheet">
<!-- Bootstrap core CSS -->
<!-- <link href="bootstrap.min.css" rel="stylesheet"> -->
<!-- Custom styles for this template -->
<link href="//maxcdn.bootstrapcdn.com/font-awesome/4.6.1/css/font-awesome.min.css" rel="stylesheet">
<link href="https://communitywatercenter.nationbui

In [21]:
#Writing to CSV file (basic)

for count in range(0, len(pagelinks)):
    
    page_url = "https://www.communitywatercenter.org" + pagelinks[count]
    page = urlopen(page_url)
    soup = BeautifulSoup(page, 'html.parser')
    title = soup.title
    date = soup.find_all(string=re.compile("Published on"))
    
    with open(file_name + '.csv', 'a') as mycsv: ## a for append
        
        writer = csv.writer(mycsv)
        
        for term in range(0, len(terms_of_interest)):
        
            phrase = soup.find_all(string=re.compile(terms_of_interest[term]))
        
            for item in range(0, len(phrase)):
            
                writer.writerow([phrase[item], str(date)[19:-8], title.string, page_url])

In [148]:
# Need to loop through list of words and print every instance with that word
# Need to recognize county names
# Need to recognize time of publication

'Además, en este mes estamos lanzando '

In [195]:
type(sites)

bs4.element.ResultSet